# Laboratorio 1
**Redes Neuronales para Lenguaje Natural, 2025**

En este laboratorio construiremos analizadores de sentimiento de tres clases (positivo, negativo, neutro) en textos que muestran opiniones, con el objetivo de comparar distintas técnicas de representación de los textos y utilizarlas para alimentar un modelo de redes neuronales.

**Entrega: 9 de octubre de 2025**

**Formato: notebook de Python (.ipynb)**

**No olvidar mantener todas las salidas de cada región de código en el notebook!**

---



## 0. Preparación del entorno

Comenzamos con algunos imports y definiciones.

In [ ]:
! pip install --upgrade gensim

import torch
import gensim
import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix

POLARITY_LABELS = ['Neg','Pos','Neu']
POLARITY_ID = {p:i for (i,p) in enumerate(POLARITY_LABELS) }

In [ ]:
POLARITY_ID

Descargamos los datos de textos de prensa, anotados con una de las siguientes clases: Pos, Neg,Neu. Por más información sobre el dataset, consulte [aquí](https://github.com/pln-fing-udelar/pln-inco-resources/tree/master/sentiment/corpusAnalisisSentimientosEsp)

In [ ]:
! wget https://raw.githubusercontent.com/pln-fing-udelar/pln-inco-resources/master/sentiment/corpusAnalisisSentimientosEsp/prensaUyUnaClase.csv

Cargamos el dataset en un dataframe de pandas y generamos conjuntos de entrenamiento, desarrollo y testeo.
Este dataset está compuesto por textos con opiniones. Para cada opinión tenemos el texto, un campo que no utilizaremos, y una categoría, que puede ser: Pos, Neg, o Neutro (indicando su polaridad).

In [ ]:
from sklearn.model_selection import train_test_split

dataset_df = pd.read_csv('./prensaUyUnaClase.csv', header=None)
dataset_df.columns = ['text', 'extra', 'polarity']

# Dividimos train y test
temp_df, test_df = train_test_split(dataset_df, test_size=0.2, random_state=42)

# Separamos conjunto de validación
train_df, dev_df = train_test_split(temp_df, test_size=0.2, random_state=42)

print("Training set size:", len(train_df))
print("Development set size:", len(dev_df))
print("Test set size:", len(test_df))

display(dataset_df.iloc[0].text)
display(dataset_df.iloc[0].polarity)





En este laboratorio utilizaremos el texto como entrada (columna "text"), y el valor a predecir será la polaridad (columna "polarity").

Imprimimos algunos textos de ejemplo y sus categorías, y luego imprimimos la cantidad de ejemplos de las tres clases en las tres particiones.

In [ ]:
train_text = train_df.loc[:,'text'].to_numpy()
dev_text = dev_df.loc[:,'text'].to_numpy()
test_text = test_df.loc[:,'text'].to_numpy()

train_labels = np.array([POLARITY_ID[l] for l in train_df.loc[:,'polarity']])
dev_labels = np.array([POLARITY_ID[l] for l in dev_df.loc[:,'polarity']])
test_labels = np.array([POLARITY_ID[l] for l in test_df.loc[:,'polarity']])

print(train_text[121])
print(POLARITY_LABELS[train_labels[121]])

print(train_text[27])
print(POLARITY_LABELS[train_labels[27]])

print(train_text[755])
print(POLARITY_LABELS[train_labels[755]])

print(*POLARITY_LABELS, sep='\t')
print(*np.bincount(train_labels), sep='\t')
print(*np.bincount(dev_labels), sep='\t')
print(*np.bincount(test_labels), sep='\t')


In [ ]:
print(train_text[1])
print(train_labels[1])

## Parte 1

Utilizando pytorch, construir un clasificador tipo Multi-Layered Perceptron (MLP) que clasifique los textos según su sentimiento.
En esta parte, se debe utilizar una representación tipo **bag-of-words (BOW)** para los tweets, la cual se utilizará como entrada de la red.

Puede probar realizar diferentes alternativas dentro de la representación BOW, por ejemplo considerar o no mayúsculas y minúsculas, descartar palabras con listas de stop words, usar N-gramas de orden mayor en vez de palabras simples, o utilizar tf-idf

**Sugerencias:**
* Utilizar la clase CountVectorizer de sklearn
* Limitar el número máximo de features para no quedarse sin memoria
* Reducir la dimensionalidad del vector utilizando, por ejemplo, SVD

Describa las características de su mejor modelo, incluyendo arquitectura e hiperparámetros. Evalúelo sobre el corpus de **dev**, imprimiendo la métrica accuracy, y las métricas macro-precision, macro-recall y macro-F1. Incluya también una matriz de confusión.

---



In [ ]:
# su código aquí


**Descripción del modelo**:

...

## Parte 2

Utilizando pytorch, construir un clasificador tipo Multi-Layered Perceptron (MLP) que clasifique los textos según su polaridad.
En esta parte, se debe utilizar una representación tipo **centroide de word embeddings** para los tweets, la cual se utilizará como entrada de la red.

Puede probar realizar diferentes alternativas dentro de la representación con word embeddings, por ejemplo considerar o no mayúsculas y minúsculas, o descartar palabras con listas de stop words.

**Sugerencias:**
* Puede utilizar uno de los embeddings que vimos en el taller en clase
* O puede bajar alguna otra colección de embeddings, pero no entrene sus propios embeddings!

## 2.1 Configuración y setup

In [ ]:
import random
import numpy as np
import torch
from torch import nn

from torch.utils.data import Dataset, DataLoader
from gensim.models import KeyedVectors, fasttext
from gensim.utils import simple_preprocess
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix
from typing import List, Dict, Optional
from sklearn.feature_extraction.text import TfidfVectorizer

# Reproducibilidad
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Usar GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Forzando la conversión a string por las dudas
train_text = np.array(train_text, dtype=str)
dev_text   = np.array(dev_text, dtype=str)
test_text  = np.array(test_text, dtype=str)

# Forzando la conversión a enteros por las dudas
train_labels = np.asarray(train_labels, dtype=np.int64)
dev_labels   = np.asarray(dev_labels, dtype=np.int64)
test_labels  = np.asarray(test_labels, dtype=np.int64)


## 2.2 Tokenizar y vectorizar datasets

In [ ]:
# -----------------------------
# 1) Tokenizador simple
# -----------------------------
def tokenize(text: str) -> List[str]:
    # minúsculas + sin signos, tokens de 2..30 chars
    return simple_preprocess(str(text), deacc=True, min_len=2, max_len=30)

In [37]:

# ------------------------------------
# Descargar ebeddings y cargar modelo
# ------------------------------------
# 2) Cargar embeddings *preentrenados* (NO entrenar nada)
# Usamos SBWC (más liviano, pero sin OOV):
!wget https://cs.famaf.unc.edu.ar/~ccardellino/SBWCE/SBW-vectors-300-min5.bin.gz
!gunzip SBW-vectors-300-min5.bin.gz
kv = KeyedVectors.load_word2vec_format("SBW-vectors-300-min5.bin", binary=True)


embed_dim = kv.vector_size

--2025-10-05 15:43:26--  https://cs.famaf.unc.edu.ar/~ccardellino/SBWCE/SBW-vectors-300-min5.bin.gz
Resolving cs.famaf.unc.edu.ar (cs.famaf.unc.edu.ar)... 200.16.17.55
Connecting to cs.famaf.unc.edu.ar (cs.famaf.unc.edu.ar)|200.16.17.55|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1123304474 (1.0G) [application/x-gzip]
Saving to: ‘SBW-vectors-300-min5.bin.gz’

SBW-vectors-300-min 100%[===================>]   1.05G  16.1MB/s    in 58s     

2025-10-05 15:44:25 (18.6 MB/s) - ‘SBW-vectors-300-min5.bin.gz’ saved [1123304474/1123304474]

gzip: SBW-vectors-300-min5.bin already exists; do you wish to overwrite (y or n)? y



In [38]:
# -----------------------------
# 3) TF-IDF para ponderar el centroide
# -----------------------------
# tfidf = TfidfVectorizer(tokenizer=tokenize, lowercase=False, min_df=2)  # ajusta min_df si el corpus es pequeño
# tfidf.fit(train_text)
tfidf = TfidfVectorizer(tokenizer=tokenize, lowercase=False, min_df=2)
tfidf.fit(train_text)

idf_vals = tfidf.idf_
vocab = tfidf.vocabulary_
idf = {t: idf_vals[i] for t, i in vocab.items()}

# Diccionario token -> idf
idf_vals = tfidf.idf_
vocab = tfidf.vocabulary_            # token -> idx
idf: Dict[str, float] = {t: idf_vals[i] for t, i in vocab.items()}


/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [39]:
# -----------------------------
# 4) Centroide (promedio ponderado por TF-IDF)
# -----------------------------
def doc_centroid(tokens, normalize: bool = True):
    # pesos TF-IDF (fallback 1.0 si no está en el vocab del TF-IDF)
    w = [idf.get(t, 1.0) for t in tokens]
    try:
        # pre_normalize/post_normalize controlan normalizaciones internas
        vec = kv.get_mean_vector(tokens, weights=w,
                                 pre_normalize=False,
                                 post_normalize=False)
    except KeyError:
        # (por si alguna versión lanza error con OOV)
        vec = np.zeros(embed_dim, dtype=np.float32)

    if normalize:
        n = np.linalg.norm(vec)
        if n > 0:
            vec = vec / n
    return vec.astype(np.float32)


def vectorize_texts(texts: List[str]) -> np.ndarray:
    return np.vstack([doc_centroid(tokenize(t)) for t in texts])

# -----------------------------
# 5) Vectorizar splits
# -----------------------------
X_train = vectorize_texts(train_text)
X_dev   = vectorize_texts(dev_text)
X_test  = vectorize_texts(test_text)

print()
print(X_train.shape)
print(X_dev.shape)
print(X_test.shape)


(806, 300)
(202, 300)
(253, 300)


## 2.3 Entrenamiento de embeddings

In [40]:
# Escalamos (fit en train, transform en dev/test) — ayuda a MLP
scaler = StandardScaler(with_mean=True, with_std=True)
X_train = scaler.fit_transform(X_train).astype(np.float32)
X_dev   = scaler.transform(X_dev).astype(np.float32)
X_test  = scaler.transform(X_test).astype(np.float32)

In [41]:
# ====== Datasets y DataLoaders ======
class NpDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X).float()
        self.y = torch.from_numpy(y).long()
    def __len__(self): return len(self.y)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

In [42]:
batch_size = 64
train_ds = NpDataset(X_train, train_labels)
dev_ds   = NpDataset(X_dev, dev_labels)
test_ds  = NpDataset(X_test, test_labels)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, drop_last=False)
dev_loader   = DataLoader(dev_ds, batch_size=batch_size, shuffle=False, drop_last=False)
test_loader  = DataLoader(test_ds, batch_size=batch_size, shuffle=False, drop_last=False)

In [43]:
# ====== MLP ======
class MLP(nn.Module):
    def __init__(self, input_dim, hidden=(256, 128), num_classes=3, dropout=0.3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden[0]),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden[0], hidden[1]),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden[1], num_classes),
        )
    def forward(self, x): return self.net(x)

model = MLP(input_dim=embed_dim, hidden=(256,128), num_classes=len(POLARITY_LABELS), dropout=0.3).to(device)

In [44]:
# Ponderación de clases (por si hay desbalance)
counts = np.bincount(train_labels, minlength=len(POLARITY_LABELS))
inv_freq = 1.0 / np.clip(counts, 1, None)
class_weights = torch.tensor(inv_freq / inv_freq.sum() * len(inv_freq), dtype=torch.float32, device=device)

criterion = nn.CrossEntropyLoss(weight=class_weights)  # ponderado
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

In [45]:
# ====== Entrenamiento con early-stopping ======
best_dev_loss = float("inf")
best_state = None
patience = 7
epochs = 50
best_epoch = -1
no_improve = 0

def run_epoch(dloader, train_mode=True):
    if train_mode:
        model.train()
    else:
        model.eval()
    total_loss = 0.0
    total_correct = 0
    total = 0
    for Xb, yb in dloader:
        Xb = Xb.to(device)
        yb = yb.to(device)
        if train_mode:
            optimizer.zero_grad()
        with torch.set_grad_enabled(train_mode):
            logits = model(Xb)
            loss = criterion(logits, yb)
            if train_mode:
                loss.backward()
                optimizer.step()
        total_loss += loss.item() * yb.size(0)
        preds = torch.argmax(logits, dim=1)
        total_correct += (preds == yb).sum().item()
        total += yb.size(0)
    return total_loss / total, total_correct / total

for epoch in range(1, epochs+1):
    tr_loss, tr_acc = run_epoch(train_loader, train_mode=True)
    dv_loss, dv_acc = run_epoch(dev_loader, train_mode=False)

    if dv_loss < best_dev_loss - 1e-5:
        best_dev_loss = dv_loss
        best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
        best_epoch = epoch
        no_improve = 0
    else:
        no_improve += 1

    print(f"Epoch {epoch:02d} | Train loss {tr_loss:.4f} acc {tr_acc:.4f} | Dev loss {dv_loss:.4f} acc {dv_acc:.4f}")
    if no_improve >= patience:
        print("Early stopping.")
        break

Epoch 01 | Train loss 1.0593 acc 0.4293 | Dev loss 1.0119 acc 0.5099
Epoch 02 | Train loss 0.8975 acc 0.6179 | Dev loss 0.9446 acc 0.5396
Epoch 03 | Train loss 0.7407 acc 0.6836 | Dev loss 0.9325 acc 0.5743
Epoch 04 | Train loss 0.6052 acc 0.7531 | Dev loss 0.9573 acc 0.5693
Epoch 05 | Train loss 0.4953 acc 0.7829 | Dev loss 1.0085 acc 0.5495
Epoch 06 | Train loss 0.3813 acc 0.8598 | Dev loss 1.1487 acc 0.5842
Epoch 07 | Train loss 0.2901 acc 0.8995 | Dev loss 1.2214 acc 0.5495
Epoch 08 | Train loss 0.1964 acc 0.9541 | Dev loss 1.3011 acc 0.5693
Epoch 09 | Train loss 0.1586 acc 0.9541 | Dev loss 1.4556 acc 0.5297
Epoch 10 | Train loss 0.1038 acc 0.9739 | Dev loss 1.5905 acc 0.5396
Early stopping.


In [46]:
# Cargamos el mejor estado
print(f'best_epoch = {best_epoch}')
if best_state is not None:
    model.load_state_dict(best_state)
model.to(device)

best_epoch = 3


MLP(
  (net): Sequential(
    (0): Linear(in_features=300, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=256, out_features=128, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=128, out_features=3, bias=True)
  )
)

In [47]:
# ====== Evaluación final (dev y test) ======
def predict_all(dloader):
    model.eval()
    preds_list, y_list = [], []
    with torch.no_grad():
        for Xb, yb in dloader:
            Xb = Xb.to(device)
            logits = model(Xb)
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            preds_list.append(preds)
            y_list.append(yb.numpy())
    y_true = np.concatenate(y_list)
    y_pred = np.concatenate(preds_list)
    return y_true, y_pred

def report_split(name, y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    pr, rc, f1, _ = precision_recall_fscore_support(y_true, y_pred, labels=list(range(len(POLARITY_LABELS))), zero_division=0)
    cm = confusion_matrix(y_true, y_pred, labels=list(range(len(POLARITY_LABELS))))
    print(f"\n== {name} ==")
    print(f"Accuracy: {acc:.4f}")
    print("Per-class (Neg, Pos, Neu) -> precision | recall | f1")
    for i, lab in enumerate(POLARITY_LABELS):
        print(f"{lab:>3}: {pr[i]:.3f} | {rc[i]:.3f} | {f1[i]:.3f}")
    print("Confusion Matrix (rows=true, cols=pred):")
    print(cm)

y_dev_true,  y_dev_pred  = predict_all(dev_loader)
y_test_true, y_test_pred = predict_all(test_loader)

report_split("DEV",  y_dev_true,  y_dev_pred)
report_split("TEST", y_test_true, y_test_pred)



== DEV ==
Accuracy: 0.5743
Per-class (Neg, Pos, Neu) -> precision | recall | f1
Neg: 0.635 | 0.659 | 0.647
Pos: 0.571 | 0.581 | 0.576
Neu: 0.481 | 0.448 | 0.464
Confusion Matrix (rows=true, cols=pred):
[[54 13 15]
 [13 36 13]
 [18 14 26]]

== TEST ==
Accuracy: 0.6047
Per-class (Neg, Pos, Neu) -> precision | recall | f1
Neg: 0.735 | 0.632 | 0.679
Pos: 0.620 | 0.550 | 0.583
Neu: 0.440 | 0.627 | 0.517
Confusion Matrix (rows=true, cols=pred):
[[72 17 25]
 [14 44 22]
 [12 10 37]]


## Observaciones:
Aprende muy rápido lo poco que puede generalizar y después empieza a empeorar (overfitting o mal ajuste de hiperparámetros).


Describa las características de su mejor modelo, incluyendo arquitectura e hiperparámetros. Evalúelo sobre el corpus de **dev**, imprimiendo la métrica accuracy, y las métricas macro-precision, macro-recall y macro-F1. Incluya también una matriz de confusión.


In [48]:
# su código aquí


# Descripción del modelo:

```
Texto ──tokenize──┐
                  ├─ TF-IDF (IDF por token)
                  └─ fastText vectors
                         │
            (promedio ponderado por IDF)
                         │
                  Centroide (300,)
                         │
                StandardScaler
                         │
                      (B,300)
                         │
        ┌────────── MLP (entrenable) ──────────┐
        │ Linear 300→256 → ReLU → Dropout      │
        │ Linear 256→128 → ReLU → Dropout      │
        │ Linear 128→3                         │
        └──────────────────────────────────────┘
                         │
                      Logits
                         │
              CrossEntropyLoss (+ class weights)
```

- Representación (centroide de embeddings): cada texto se transforma en el promedio de los vectores Word2Vec de sus palabras presentes en el vocabulario. Es simple, rápido y funciona sorprendentemente bien como baseline.
- MLP: dos capas ocultas (256→128), ReLU, Dropout=0.3, Adam (1e-3), weight_decay ligero (1e-4).
- Desbalance: se calculan pesos por clase a partir de la frecuencia en train para CrossEntropyLoss(weight=...).
- Entrenamiento: early-stopping por dev_loss con patience=7.
- Escalado: StandardScaler (fit en train) mejora la estabilidad del MLP.

Repita el paso anterior, utilizando la misma arquitectura, pero cambiando los embeddings utilizados. Compare los resultados y comente.

In [ ]:
# 2.BIS)
# Otras opciones:
# - FASTTEXT (mejor para tweets con OOV, pero pesado)
# - SBWC (más liviano, pero sin OOV)
# -----------------------------
# Alaternativa A (recomendada pero consume mucha RAM): fastText .bin con OOV
# kv = fasttext.load_facebook_vectors("cc.es.300.bin")  # FastTextKeyedVectors

# Alternativa B (más liviana): Word2Vec/GloVe en formato word2vec .vec SIN OOV
! wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.es.300.vec.gz
! gunzip cc.es.300.vec.gz
kv = KeyedVectors.load_word2vec_format("cc.es.300.vec", binary=False)

# -----------------------------
# 3.BIS) TF-IDF para ponderar el centroide
# -----------------------------
# tfidf = TfidfVectorizer(tokenizer=tokenize, lowercase=False, min_df=2)  # ajusta min_df si el corpus es pequeño
# tfidf.fit(train_text)
tfidf = TfidfVectorizer(tokenizer=tokenize, lowercase=False, min_df=2)
tfidf.fit(train_text)

idf_vals = tfidf.idf_
vocab = tfidf.vocabulary_
idf = {t: idf_vals[i] for t, i in vocab.items()}

# Diccionario token -> idf
idf_vals = tfidf.idf_
vocab = tfidf.vocabulary_            # token -> idx
idf: Dict[str, float] = {t: idf_vals[i] for t, i in vocab.items()}

# -----------------------------
# 4.BIS) Centroide (promedio ponderado por TF-IDF)
# -----------------------------
def doc_centroid(tokens, normalize: bool = True):
    # pesos TF-IDF (fallback 1.0 si no está en el vocab del TF-IDF)
    w = [idf.get(t, 1.0) for t in tokens]
    try:
        # pre_normalize/post_normalize controlan normalizaciones internas
        vec = kv.get_mean_vector(tokens, weights=w,
                                 pre_normalize=False,
                                 post_normalize=False)
    except KeyError:
        # (por si alguna versión lanza error con OOV)
        vec = np.zeros(embed_dim, dtype=np.float32)

    if normalize:
        n = np.linalg.norm(vec)
        if n > 0:
            vec = vec / n
    return vec.astype(np.float32)


def vectorize_texts(texts: List[str]) -> np.ndarray:
    return np.vstack([doc_centroid(tokenize(t)) for t in texts])

# -----------------------------
# 5.BIS) Vectorizar splits
# -----------------------------
X_train = vectorize_texts(train_text)
X_dev   = vectorize_texts(dev_text)

print()
print(X_train.shape)
print(X_dev.shape)

# -----------------------------
#  Entrenamiento de embeddings
# -----------------------------

# ====== Datasets y DataLoaders ======
class NpDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X).float()
        self.y = torch.from_numpy(y).long()
    def __len__(self): return len(self.y)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

batch_size = 64
train_ds = NpDataset(X_train, train_labels)
dev_ds   = NpDataset(X_dev, dev_labels)
test_ds  = NpDataset(X_test, test_labels)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, drop_last=False)
dev_loader   = DataLoader(dev_ds, batch_size=batch_size, shuffle=False, drop_last=False)
test_loader  = DataLoader(test_ds, batch_size=batch_size, shuffle=False, drop_last=False)

# ====== MLP ======
class MLP(nn.Module):
    def __init__(self, input_dim, hidden=(256, 128), num_classes=3, dropout=0.3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden[0]),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden[0], hidden[1]),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden[1], num_classes),
        )
    def forward(self, x): return self.net(x)

model = MLP(input_dim=embed_dim, hidden=(256,128), num_classes=len(POLARITY_LABELS), dropout=0.3).to(device)

# Ponderación de clases (por si hay desbalance)
counts = np.bincount(train_labels, minlength=len(POLARITY_LABELS))
inv_freq = 1.0 / np.clip(counts, 1, None)
class_weights = torch.tensor(inv_freq / inv_freq.sum() * len(inv_freq), dtype=torch.float32, device=device)

criterion = nn.CrossEntropyLoss(weight=class_weights)  # ponderado
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

# ====== Entrenamiento con early-stopping ======
best_dev_loss = float("inf")
best_state = None
patience = 7
epochs = 50
best_epoch = -1
no_improve = 0

def run_epoch(dloader, train_mode=True):
    if train_mode:
        model.train()
    else:
        model.eval()
    total_loss = 0.0
    total_correct = 0
    total = 0
    for Xb, yb in dloader:
        Xb = Xb.to(device)
        yb = yb.to(device)
        if train_mode:
            optimizer.zero_grad()
        with torch.set_grad_enabled(train_mode):
            logits = model(Xb)
            loss = criterion(logits, yb)
            if train_mode:
                loss.backward()
                optimizer.step()
        total_loss += loss.item() * yb.size(0)
        preds = torch.argmax(logits, dim=1)
        total_correct += (preds == yb).sum().item()
        total += yb.size(0)
    return total_loss / total, total_correct / total

for epoch in range(1, epochs+1):
    tr_loss, tr_acc = run_epoch(train_loader, train_mode=True)
    dv_loss, dv_acc = run_epoch(dev_loader, train_mode=False)

    if dv_loss < best_dev_loss - 1e-5:
        best_dev_loss = dv_loss
        best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
        best_epoch = epoch
        no_improve = 0
    else:
        no_improve += 1

    print(f"Epoch {epoch:02d} | Train loss {tr_loss:.4f} acc {tr_acc:.4f} | Dev loss {dv_loss:.4f} acc {dv_acc:.4f}")
    if no_improve >= patience:
        print("Early stopping.")
        break

# Cargamos el mejor estado
print(f'best_epoch = {best_epoch}')
if best_state is not None:
    model.load_state_dict(best_state)
model.to(device)

# ====== Evaluación final (dev y test) ======
def predict_all(dloader):
    model.eval()
    preds_list, y_list = [], []
    with torch.no_grad():
        for Xb, yb in dloader:
            Xb = Xb.to(device)
            logits = model(Xb)
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            preds_list.append(preds)
            y_list.append(yb.numpy())
    y_true = np.concatenate(y_list)
    y_pred = np.concatenate(preds_list)
    return y_true, y_pred

def report_split(name, y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    pr, rc, f1, _ = precision_recall_fscore_support(y_true, y_pred, labels=list(range(len(POLARITY_LABELS))), zero_division=0)
    cm = confusion_matrix(y_true, y_pred, labels=list(range(len(POLARITY_LABELS))))
    print(f"\n== {name} ==")
    print(f"Accuracy: {acc:.4f}")
    print("Per-class (Neg, Pos, Neu) -> precision | recall | f1")
    for i, lab in enumerate(POLARITY_LABELS):
        print(f"{lab:>3}: {pr[i]:.3f} | {rc[i]:.3f} | {f1[i]:.3f}")
    print("Confusion Matrix (rows=true, cols=pred):")
    print(cm)

y_dev_true,  y_dev_pred  = predict_all(dev_loader)
y_test_true, y_test_pred = predict_all(test_loader)

report_split("DEV",  y_dev_true,  y_dev_pred)
report_split("TEST", y_test_true, y_test_pred)


--2025-10-05 15:53:03--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.es.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 99.84.41.80, 99.84.41.79, 99.84.41.129, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|99.84.41.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1285580896 (1.2G) [binary/octet-stream]
Saving to: ‘cc.es.300.vec.gz’

cc.es.300.vec.gz    100%[===================>]   1.20G   221MB/s    in 5.0s    

2025-10-05 15:53:08 (244 MB/s) - ‘cc.es.300.vec.gz’ saved [1285580896/1285580896]

gzip: cc.es.300.vec already exists; do you wish to overwrite (y or n)? 

**Comentarios**:
La alternativa A no la pudimos probar porque consume más RAM de la que tenemos disponible para el laboratorio.

TOOD:  comparar
...

## Parte 3 (opcional):
Elija uno de los clasificadores anteriores y realice una búsqueda automatizada de hiperparámetros del modelo. Por ejemplo, puede utilizar una búsqueda en grilla completa o una búsqueda aleatoria, eligiendo diferente cantidad de capas, número unidades por capa, funciones de activación, y valores de learning rate. Intente encontrar el mejor clasificador posible, comparándolos sobre el corpus de **dev**.

In [ ]:
# su código aquí


Describa las características de su mejor modelo, incluyendo arquitectura e hiperparámetros. Evalúelo sobre el corpus de **dev**, imprimiendo la métrica accuracy, y las métricas macro-precision, macro-recall y macro-F1. Incluya también una matriz de confusión.

In [ ]:
# su código aquí


## Parte 4:

Elija el mejor clasificador de todos los construidos en las partes anteriores, comparándolos utilizando la medida macro-F1, y evalúelo sobre el corpus de **test**, imprimiendo la métrica accuracy, y las métricas macro-precision, macro-recall y macro-F1.

In [ ]:
# su código aquí


Despliege la matriz de confusión sobre el corpus de **test** para las tres clases (P, N, NEU) para el mejor clasificador construido en las partes anteriores.

In [ ]:
# su código aquí


Analice los resultados, y responda, al menos, las siguientes preguntas:

1.   ¿Cuál fue la representación que funcionó mejor? Describa brevemente sus características.
2.   ¿Cuál es la forma del modelo? Indique cantidad de capas, unidades, y funciones de activación. Puede usar el método print() sobre el modelo para imprimir la forma del modelo.
3.   Indique qué categoría o categorías fueron las más difíciles de aprender para su clasificador. ¿La categoría más difícil es la misma para el mejor clasificador de las partes 1 y 2 (y la parte 3 si la hizo)?

( sus respuestas aquí)
